# **下载数据集**

In [1]:
! wget https://www.dropbox.com/s/6l2vcvxl54b0b6w/food11.zip
! unzip food11.zip

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: food11/training/8_806.jpg  
  inflating: food11/training/8_807.jpg  
  inflating: food11/training/8_808.jpg  
  inflating: food11/training/8_809.jpg  
  inflating: food11/training/8_81.jpg  
  inflating: food11/training/8_810.jpg  
  inflating: food11/training/8_811.jpg  
  inflating: food11/training/8_812.jpg  
  inflating: food11/training/8_813.jpg  
  inflating: food11/training/8_814.jpg  
  inflating: food11/training/8_815.jpg  
  inflating: food11/training/8_816.jpg  
  inflating: food11/training/8_817.jpg  
  inflating: food11/training/8_818.jpg  
  inflating: food11/training/8_819.jpg  
  inflating: food11/training/8_82.jpg  
  inflating: food11/training/8_820.jpg  
  inflating: food11/training/8_821.jpg  
  inflating: food11/training/8_822.jpg  
  inflating: food11/training/8_823.jpg  
  inflating: food11/training/8_824.jpg  
  inflating: food11/training/8_825.jpg  
  inflating: food11/training/8_826.jpg  
  inflating: food11/training/8_8

# If you want to access the version you have already modified, click "Edit"
# If you want to access the original sample code, click "...", then click "Copy & Edit Notebook"

In [2]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
_exp_name = "sample"

In [4]:
# Import necessary packages.
import numpy as np
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [5]:
#设置随机数种子
myseed = 6666  # set a random seed for reproducibility
#使用确定性算法
torch.backends.cudnn.deterministic = True
#cuDNN的benchmark模式自动寻找最快的卷积实现方法
torch.backends.cudnn.benchmark = False
#固定numpy和pytorch和GPU的随机种子
np.random.seed(myseed)
torch.manual_seed(myseed)
#即使多GPU每个GPU的随机性都一致
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

## **Transforms**---图像预处理
*Torchvision* provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [6]:
#在dataloader读入数据前对图像进行转换
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
'''test_tfm = transforms.Compose([#将以下流水线串联
    transforms.Resize((128, 128)),#将图片固定大小
    transforms.ToTensor(),#转换为PyTorch 张量(C×H×W）
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])'''
from torchvision import transforms

# 先用 ImageNet 的均值方差\
MEAN = [0.555, 0.451, 0.343]
STD  = [0.225, 0.235, 0.234]

# ✅ 验证/测试：确定性（不随机）
test_tfm = transforms.Compose([
    transforms.Resize(144),           # 先放大一点
    transforms.CenterCrop(128),       # 再中心裁到 128，避免直接拉伸变形
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])

# ✅ 训练：轻量增强（稳妥耐用）
train_tfm = transforms.Compose([
    transforms.Resize(144),
    transforms.RandomCrop(128),                   # 比 RRC 温和，保留主体
    transforms.RandomHorizontalFlip(p=0.5),       # 水平翻转常有益
    transforms.RandomRotation(degrees=10),        # 小角度即可，避免过强失真
    transforms.ColorJitter(0.1, 0.1, 0.05, 0.02), # 轻微颜色扰动；食物对颜色敏感，不要太大

    transforms.ToTensor(),
    # （可选）随机擦除：在 ToTensor 之后、Normalize 之前
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.08), ratio=(0.3, 3.3), value='random'),

    transforms.Normalize(MEAN, STD),
])


## **Datasets**
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [7]:

class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):

      #调用父类Dataset
        super(FoodDataset).__init__()
        #记录根目录
        self.path = path
#进行筛选        #扫描文件并排序 拼成完整路径 列出文件名 只保留jpg结尾的
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])

        self.transform = tfm


        #输出筛选后数量
    def __len__(self):
        return len(self.files)
#获取第i个的信息
    def __getitem__(self,idx):
        #第i个文件完整路径
        fname = self.files[idx]
        #读图片
        im = Image.open(fname)
        #处理图片调用到上面的.transform
        im = self.transform(im)
        #im = self.data[idx]
        try:#取文件名2_dog_001.jpg的2
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label



In [8]:
import torch
import torch.nn as nn

def conv3x3(in_ch, out_ch, stride=1):
    return nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = conv3x3(in_ch, out_ch, stride)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.relu  = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_ch, out_ch, 1)
        self.bn2   = nn.BatchNorm2d(out_ch)

        # 维度/步幅不一致时，用 1x1 卷积对齐 identity
        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_ch),
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(identity)

        out = out + identity
        out = self.relu(out)
        return out


In [9]:
#CNN分类器---处理输入图片
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]进行五次池化提取特征压缩空间
        '''self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128] 输入3通道输出64特征每个卷积核3x3 padding=1保持宽高不变
            nn.BatchNorm2d(64),#批量归一化
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0), #2x2区域取最大值再向右/下移动2 高宽减半     # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]

            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )'''
        self.cnn = nn.Sequential(
            # stem：保持 128x128
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            # layer1: 64通道，尺寸 128x128（两个残差块，第一块 stride=1）
            BasicBlock(64, 64, stride=1),
            BasicBlock(64, 64, stride=1),

            # layer2: 下采样到 64x64，通道到 128
            BasicBlock(64, 128, stride=2),
            BasicBlock(128, 128, stride=1),

            # layer3: 下采样到 32x32，通道到 256
            BasicBlock(128, 256, stride=2),
            BasicBlock(256, 256, stride=1),

            # layer4: 下采样到 16x16，通道到 512
            BasicBlock(256, 512, stride=2),
            BasicBlock(512, 512, stride=1),

            nn.AdaptiveAvgPool2d(4),
        )
        '''
        #全连接层FC
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.3), #减少过拟合
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(p=0.3), #减少过拟合
            nn.Linear(512, 11)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)
        '''

        self.gap = nn.AdaptiveAvgPool2d(1)          # [B,512,16,16] -> [B,512,1,1]
        self.fc  = nn.Sequential(
            nn.Flatten(),                            # [B,512]
            nn.Dropout(p=0.2),                       # 轻一点即可
            nn.Linear(512, 11)                       # 小头：更快更稳
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.gap(x)
        x = self.fc(x)
        return x

In [10]:
#分数据集
batch_size = 64
_dataset_dir = "./food11"#根目录
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = FoodDataset(os.path.join(_dataset_dir,"training"), tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = FoodDataset(os.path.join(_dataset_dir,"validation"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

One ./food11/training sample ./food11/training/0_0.jpg
One ./food11/validation sample ./food11/validation/0_0.jpg


In [11]:
import torch.nn.functional as F
#FocalLoss
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super().__init__()
        self.gamma = gamma
        self.reduction = reduction
        self.register_buffer('alpha', alpha if alpha is not None else None)

    def forward(self, logits, targets):
        # logits: [B, C], targets: [B]
        log_prob = F.log_softmax(logits, dim=1)
        prob = log_prob.exp()
        log_pt = log_prob.gather(1, targets.unsqueeze(1)).squeeze(1)
        pt = prob.gather(1, targets.unsqueeze(1)).squeeze(1)
        at = 1.0 if self.alpha is None else self.alpha[targets]
        loss = - at * (1 - pt).pow(self.gamma) * log_pt
        return loss.mean() if self.reduction == 'mean' else loss.sum()


In [12]:
from collections import Counter
import numpy as np
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

num_classes = 11

# 用 train_loader 扫一遍统计标签数量（只做一次，开销很小）
label_counter = Counter()
for _, ys in train_loader:
    label_counter.update(ys.tolist())

counts = np.array([label_counter.get(i, 0) for i in range(num_classes)], dtype=np.float32) + 1e-6
inv = 1.0 / counts
alpha = torch.tensor(inv * (len(counts) / inv.sum()), dtype=torch.float32).to(device)  # 归一化到均值≈1


In [13]:

# The number of training epochs and patience.训练轮数和早发触停
#n_epochs = 30
n_epochs = 25
patience = 1000 # If no improvement in 'patience' epochs, early stop

# Initialize a model, and put it on the device specified.将CNN放到处理器上
model = Classifier().to(device)

# For the classification task, we use cross-entropy as the measurement of performance.
#criterion = nn.CrossEntropyLoss()#适用于多酚类
criterion = FocalLoss(alpha=alpha, gamma=2.0)

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))#交叉熵计算loss

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()#将那张图错了为什么记录下来

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)#梯度裁剪到10以内

        # Update the parameters with computed gradients.
        optimizer.step()#优化

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():#不求梯度
            logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # update logs
    if valid_acc > best_acc:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break

  0%|          | 0/155 [00:00<?, ?it/s]

[ Train | 001/025 ] loss = 1.29802, acc = 0.21960


  0%|          | 0/54 [00:00<?, ?it/s]

[ Valid | 001/025 ] loss = 1.15792, acc = 0.30553
[ Valid | 001/025 ] loss = 1.15792, acc = 0.30553 -> best
Best model found at epoch 0, saving model


  0%|          | 0/155 [00:00<?, ?it/s]

...

[ Valid | 024/025 ] loss = 0.49972, acc = 0.66301
[ Valid | 024/025 ] loss = 0.49972, acc = 0.66301


  0%|          | 0/155 [00:00<?, ?it/s]

[ Train | 025/025 ] loss = 0.33524, acc = 0.72236


  0%|          | 0/54 [00:00<?, ?it/s]

[ Valid | 025/025 ] loss = 0.37793, acc = 0.72402
[ Valid | 025/025 ] loss = 0.37793, acc = 0.72402 -> best
Best model found at epoch 24, saving model


In [17]:
test_set = FoodDataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

One ./food11/test sample ./food11/test/0001.jpg


## Testing and generate prediction CSV

In [15]:
model_best = Classifier().to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.eval()
prediction = []
with torch.no_grad():
    for data,_ in test_loader:
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()

In [16]:
#create test csv
from google.colab import files

def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("submission.csv",index = False)

files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>